In [11]:
import sys
sys.path.append('./python_modules')

import importlib

In [12]:
import init

print(f"engin:{init.engin}")

engin:azure-gpt-4o


In [3]:
from plot_module import greet
greet('Good morning')
 

'Hello, Good morning!'

In [4]:
from llama_index.legacy.multi_modal_llms.azure_openai import AzureOpenAIMultiModal
from llama_index.core import SimpleDirectoryReader, StorageContext

In [5]:
file_path="./sql_files"

In [6]:
documents = SimpleDirectoryReader(file_path).load_data()
print(len(documents))
from llama_index.core import Document, VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)

2


In [7]:
import openai

client = openai.OpenAI(
    api_key=init.api_key,#api_key, # Format should be 'sk-<your_key>'
    base_url=init.azure_endpoint #"http://litellm.toxpipe.niehs.nih.gov" # LiteLLM Proxy is OpenAI compatible, Read More: https://docs.litellm.ai/docs/proxy/user_keys
)

response = client.chat.completions.create(
    model=init.engin,#"azure-gpt-4o", # model to send to the proxy
    messages = [
        {
            "role": "user",
            #"content": "this is a test request, write a short poem"
             "content": f"Summarize the following text:\n {documents}",
        }
    ]
)

#print(response.choices[0].message.content)
#print(response)

In [8]:
print(response.choices[0].message.content)

The document contains SQL statements for creating various tables in a database. These tables include:

1. `devices_data`: Stores information about devices with fields like ID, source ID, well details, device details, and other properties.
2. `endpoints`: Contains endpoint details including ID, name, and description.
3. `endpoint_data`: Connects endpoints to devices and stores endpoint values, referencing the `endpoints` and `devices_data` tables.
4. `sources`: Stores source details with fields for ID, name, and code.
5. `chemicals`: Includes chemical information such as chemical ID, code, DTXSID, CASRN, and preferred name.
6. `well_types`: Details about well types, including ID, type, and description.
7. `plates`: Contains details about plates, associating them with sources and including fields like plate ID, name, screening time, and stage.
8. `bins`: Stores bin ranges identified by bin ID.
9. `phases`: Represents different phases with fields for ID, code, stage, and associated source

## Set method for the chatbot

In [9]:
def describe (question):
    response = client.chat.completions.create(
    model="azure-gpt-4o", # model to send to the proxy
    messages = [
        {
            "role": "user",
            #"content": "this is a test request, write a short poem"
             "content": f"{question}:\n {documents}",
             #"content": f"Summarize the following text:\n {documents}",
            
        }
    ]
)
    return response.choices[0].message.content
 

In [10]:
import gradio as gr 
def greet(name): return "Hello " + name + "!" 
def prediction (question): 
    answer = describe (question)
    return answer

#demo = gr.Interface(fn=prediction, inputs="textbox", outputs="textbox") 
demo = gr.Interface( fn=prediction,     
    inputs=gr.Textbox(lines=5, placeholder="Type your question here..."),  # Input textbox with 5 lines
    outputs=gr.Textbox(label="Answer", lines=4)  # Output textbox with 4 lines
)


if __name__ == "__main__": 
    demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7864
Running on public URL: https://a8664e93af7fec06fd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [14]:
from plot_module import greet
from plot_module import parse_sqls
from plot_module import exec_query
from plot_module import exec_query_test


In [15]:
answer_old="""
To determine the data distribution of `behavior_data` with `distance_moved` greater than 1400, we can create a SQL query to extract and analyze the relevant rows from the `behavior_data` table. Here's a step-by-step approach to achieve this:

1. **Select the Desired Data**:
    - We need to select rows from the `behavior_data` table where `distance_moved` is greater than 1400.
    - We can also join with other relevant tables if needed to gather more context or filter by additional criteria.

2. **Analyze the Distribution**:
    - Use statistical measures (mean, median, standard deviation) or visualization tools (histogram) to understand the distribution of the `distance_moved` values.

Here is an example SQL query to retrieve the necessary data:

```sql
SELECT distance_moved
FROM behavior_data
WHERE distance_moved > 1400;
```

If you want to join other tables to filter by additional criteria, you can modify the query accordingly:

```sql
SELECT bd.distance_moved, dt.chemical_id, dt.well_position 
FROM behavior_data bd
JOIN devtox_data dt ON bd.devtox_data_id = dt.devtox_data_id
WHERE bd.distance_moved > 1400;
```

To fully analyze the distribution, you would then typically use a programming language like Python with libraries such as Pandas or SQL for data processing and visualization. Below is an example in Python assuming you have the results in a DataFrame:

```python
import pandas as pd
import matplotlib.pyplot as plt

# Assuming df is the DataFrame containing your SQL query result
df = pd.read_sql(query, connection)

# Describe the statistics
print(df['distance_moved'].describe())

# Plot the histogram
plt.hist(df['distance_moved'], bins=30)
plt.title('Distribution of Distance Moved > 1400')
plt.xlabel('Distance Moved')
plt.ylabel('Frequency')
plt.show()
```

If you don't have direct access to the data and just need to understand the distribution conceptually, this outline should give you a good starting point.
"""

In [32]:
answer = """To determine the distribution of `behavior_data` with `distance_moved` greater than 1400, let's examine the table definition and some example SQL queries that could be used to calculate this distribution. 

### Table Structure
The `behavior_data` table is defined with the following columns:
- `devtox_data_id` (INTEGER)
- `phase_id` (INTEGER)
- `bin_id` (INTEGER)
- `distance_moved` (REAL)
- `time_end` (INTEGER)

This table has foreign key relationships with the `devtox_data`, `phases`, and `bins` tables. The primary key is a composite of `devtox_data_id`, `phase_id`, and `bin_id`.

### Assumptions
I'm assuming you want to understand how many rows have `distance_moved` greater than 1400 and potentially other statistics like mean, median, etc., of `distance_moved` for the relevant rows.

### Query to Retrieve Distribution

Here's a query to retrieve the count of rows in `behavior_data` where `distance_moved` is greater than 1400:

```sql
SELECT COUNT(*) as count
FROM behavior_data 
WHERE distance_moved > 1400;
```

To get detailed statistics, such as the average, minimum, and maximum values, you could use:

```sql
SELECT 
    COUNT(*) AS count,
    AVG(distance_moved) AS avg_distance,
    MIN(distance_moved) AS min_distance,
    MAX(distance_moved) AS max_distance
FROM behavior_data
WHERE distance_moved > 1400;
```

If you need a more comprehensive distribution (e.g., a histogram), you can group the distances into bins. Here's an example query that groups the `distance_moved` values into ranges of 100:

```sql
SELECT 
    FLOOR(distance_moved / 100) * 100 AS distance_bin,
    COUNT(*) AS count
FROM behavior_data
WHERE distance_moved > 1400
GROUP BY distance_bin
ORDER BY distance_bin;
```

This query will group the `distance_moved` values into bins of 100 units and provide the count of rows in each bin, starting from 1400.

### Summary
To summarize, the primary task involves querying the `behavior_data` table for rows where `distance_moved` exceeds 1400 and then applying aggregate functions or grouping to understand the data distribution. The examples provided demonstrate how you can achieve these tasks using SQL.
"""

## load database data

In [33]:
from sqlalchemy import text
from langchain_community.utilities import SQLDatabase
import pandas as pd

# Initialize the database
db_url = './data/vanna_zebrafish/OECD_ZF_LDTT.sqlite'
db = SQLDatabase.from_uri(f'sqlite:///{db_url}')

# Use the underlying SQLAlchemy engine
query = "SELECT source_name, source_code FROM sources;"
with db._engine.connect() as connection:  # Access the private _engine attribute
    result_proxy = connection.execute(text(query))  # Wrap query with `text`
    column_names = result_proxy.keys()  # Get column names
    data = result_proxy.fetchall()  # Get all rows

# Print results
print("Column Names:", column_names)
print("Data:", data)

# Convert to a Pandas DataFrame
df = pd.DataFrame(data, columns=column_names)
print(df)


Column Names: RMKeyView(['source_name', 'source_code'])
Data: [('biobide', 'a'), ('nrc', 'b'), ('osu', 'c'), ('zeclinics', 'd')]
  source_name source_code
0     biobide           a
1         nrc           b
2         osu           c
3   zeclinics           d


In [34]:
llm_output=answer
#print(parse_sqls(llm_output))#llm_output
parsed_queries = parse_sqls(llm_output)
#Output the parsed queries
#for i, query in enumerate(parsed_queries, start=1): 
    #print(f"Query {i}:\n{query}\n")
#exec_query_test(parsed_queries,db)

xxxQuery 1:
SELECT COUNT(*) as count
FROM behavior_data 
WHERE distance_moved > 1400;

xxxQuery 2:
SELECT 
    COUNT(*) AS count,
    AVG(distance_moved) AS avg_distance,
    MIN(distance_moved) AS min_distance,
    MAX(distance_moved) AS max_distance
FROM behavior_data
WHERE distance_moved > 1400;

xxxQuery 3:
SELECT 
    FLOOR(distance_moved / 100) * 100 AS distance_bin,
    COUNT(*) AS count
FROM behavior_data
WHERE distance_moved > 1400
GROUP BY distance_bin
ORDER BY distance_bin;



In [35]:
import re
print(llm_output)
#sql_queries = re.findall(r"```SQL(.*?)```", llm_output, re.DOTALL | re.IGNORECASE)
#sql_queries = re.findall(r"```sql(.*?)```", llm_output, re.DOTALL | re.IGNORECASE)
sql_queries = re.findall(r"(?:sql|SQL)(.*?)", llm_output, re.DOTALL)

To determine the distribution of `behavior_data` with `distance_moved` greater than 1400, let's examine the table definition and some example SQL queries that could be used to calculate this distribution. 

### Table Structure
The `behavior_data` table is defined with the following columns:
- `devtox_data_id` (INTEGER)
- `phase_id` (INTEGER)
- `bin_id` (INTEGER)
- `distance_moved` (REAL)
- `time_end` (INTEGER)

This table has foreign key relationships with the `devtox_data`, `phases`, and `bins` tables. The primary key is a composite of `devtox_data_id`, `phase_id`, and `bin_id`.

### Assumptions
I'm assuming you want to understand how many rows have `distance_moved` greater than 1400 and potentially other statistics like mean, median, etc., of `distance_moved` for the relevant rows.

### Query to Retrieve Distribution

Here's a query to retrieve the count of rows in `behavior_data` where `distance_moved` is greater than 1400:

```sql
SELECT COUNT(*) as count
FROM behavior_data 
WHERE 

In [36]:
print(parsed_queries)

['SELECT COUNT(*) as count\nFROM behavior_data \nWHERE distance_moved > 1400;', 'SELECT \n    COUNT(*) AS count,\n    AVG(distance_moved) AS avg_distance,\n    MIN(distance_moved) AS min_distance,\n    MAX(distance_moved) AS max_distance\nFROM behavior_data\nWHERE distance_moved > 1400;', 'SELECT \n    FLOOR(distance_moved / 100) * 100 AS distance_bin,\n    COUNT(*) AS count\nFROM behavior_data\nWHERE distance_moved > 1400\nGROUP BY distance_bin\nORDER BY distance_bin;']


In [37]:
results = []
results = exec_query(parsed_queries,db)

SELECT COUNT(*) as count
FROM behavior_data 
WHERE distance_moved > 1400;
exec query 1 : SELECT COUNT(*) as count
FROM behavior_data 
WHERE distance_moved > 1400;
Column Names: RMKeyView(['count'])
Data: [(334,)]
column lenth : 1
exec query 2 : SELECT 
    COUNT(*) AS count,
    AVG(distance_moved) AS avg_distance,
    MIN(distance_moved) AS min_distance,
    MAX(distance_moved) AS max_distance
FROM behavior_data
WHERE distance_moved > 1400;
Column Names: RMKeyView(['count', 'avg_distance', 'min_distance', 'max_distance'])
Data: [(334, 2191.708718562874, 1407.23, 2958.85)]
column lenth : 4
exec query 3 : SELECT 
    FLOOR(distance_moved / 100) * 100 AS distance_bin,
    COUNT(*) AS count
FROM behavior_data
WHERE distance_moved > 1400
GROUP BY distance_bin
ORDER BY distance_bin;
Column Names: RMKeyView(['distance_bin', 'count'])
Data: [(1400, 10), (1500, 11), (1600, 11), (1700, 16), (1800, 13), (1900, 28), (2000, 25), (2100, 43), (2200, 47), (2300, 42), (2400, 34), (2500, 19), (2600, 15

In [38]:
for i, result in enumerate(results, start=1):
    print (F"Results : {result}")

Results :    count
0    334
Results :    count  avg_distance  min_distance  max_distance
0    334   2191.708719       1407.23       2958.85
Results :     distance_bin  count
0           1400     10
1           1500     11
2           1600     11
3           1700     16
4           1800     13
5           1900     28
6           2000     25
7           2100     43
8           2200     47
9           2300     42
10          2400     34
11          2500     19
12          2600     15
13          2700     11
14          2800      7
15          2900      2


## generate the plot based on the output data

In [48]:
plot_prompt = """I have a dataset and want to generate Plotly visualization code based on the data I passed in. The dataset is represented as a Pandas DataFrame.

Here are the requirements:

Use the provided dataset (data) to create a Plotly figure.
Use the go.Scatter trace type.
The x-axis , the y-axis .
Include a hover template displaying the x  and y values.
Set a custom marker color and symbol (e.g., blue circle).
Include x-axis and y-axis titles in the layout.
Use the Plotly template='plotly' for styling.
Output the Python code in the following format:

 
```python 
data = pd.DataFrame({...})  # Example dataset

# Create the Plotly Figure
figure = go.Figure(
    data=[
       
    ],
    layout=dict(
         
        template='plotly'
    )
)
figue.show()
"""

In [73]:
plot_prompt_2 = """Can you use it to generate Plotly visualization code? The dataset is represented as a Pandas DataFrame.

Here are the requirements:

Use the provided dataset (data) to create a Plotly figure.
 
The x-axis , the y-axis .
Include a hover template displaying the x  and y values.
Set a custom marker color and symbol (e.g., blue circle).
Include x-axis and y-axis titles in the layout.
choose the best plot type for the data for instance  'bar', 'pie' or 'Scatter' whichever is properly for the data
Use the Plotly template='plotly' for styling.
Output the Python code in the following format:

 
```python 
import numpy as np

data = mydata  # replace this by passed in data

# Create the Plotly Figure
fig = go.Figure({
     'data': [{'domain': ,
               'hovertemplate': ,
               'labels': np.array(      ),
               'legendgroup': '',
               'name': '',
               'showlegend': True,
               'type': ,
               'values': np.array([])}],
     'layout': {'legend': {'tracegroupgap': 0}, 'template': '', 'title': {'text': ''}}
 })
fig.show()
"""

In [74]:
def draw_plot (question,data):
    response = client.chat.completions.create(
    model="azure-gpt-4o", # model to send to the proxy
    messages = [
        {
            "role": "user",
            #"content": "this is a test request, write a short poem"
             "content": f"Here is the data {data} and {question}",
             #"content": f"Summarize the following text:\n {documents}",
            
        }
    ]
)
    return response.choices[0].message.content

In [75]:
draw_plot(plot_prompt_2, results[0])

"Since you provided a simple dataset with only one count value, I believe a bar plot would be most appropriate for visualization. Here’s the Python code meeting your requirements:\n\n```python\nimport numpy as np\nimport pandas as pd\nimport plotly.graph_objects as go\n\n# Provided data\ndata = pd.DataFrame({'count': [334]})\n\n# Create the Plotly Figure\nfig = go.Figure({\n    'data': [{\n        'x': ['count'],  # x-axis value\n        'y': data['count'],  # y-axis values from the dataset\n        'type': 'bar',  # bar plot type\n        'marker': {'color': 'blue', 'symbol': 'circle'},  # custom marker\n        'hovertemplate': 'x: %{x}<br>y: %{y}<extra></extra>',  # hover template\n        'showlegend': False,  # No legend needed\n    }],\n    'layout': {\n        'xaxis': {\n            'title': 'Category',  # x-axis title\n        },\n        'yaxis': {\n            'title': 'Count',  # y-axis title\n        },\n        'template': 'plotly',  # template for styling\n        'title

In [76]:
#import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
##****important run this before run figure.show()
import plotly.io as pio
pio.renderers.default = 'iframe'
import plotly.graph_objects as go


In [83]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Provided data
data = pd.DataFrame({'count': [334]})

# Create the Plotly Figure
fig = go.Figure({
    'data': [{
        'x': ['count'],  # x-axis value
        'y': data['count'].tolist(),
        'type': 'bar',  # bar plot type
        'marker': {'color': 'blue'},  # custom marker
        'hovertemplate': 'x: %{x}<br>y: %{y}<extra></extra>',  # hover template
        'showlegend': False,  # No legend needed
    }],
    'layout': {
        'xaxis': {
            'title': 'Category',  # x-axis title
        },
        'yaxis': {
            'title': 'Count',  # y-axis title
        },
        'template': 'plotly',  # template for styling
        'title': {'text': 'Bar Plot for Count'}  # plot title
    }
})

fig.show()

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Provided data
data = pd.DataFrame({'count': [334]})

# Create the Plotly Figure
fig = go.Figure(
    data=[
        go.Bar(
            x=['count'],  # x-axis value
            y=data['count'].tolist(),  # Convert Pandas Series to list for y-axis
            marker=dict(color='blue'),  # Custom bar color
            hovertemplate='Category: %{x}<br>Count: %{y}<extra></extra>'  # Hover template
        )
    ],
    layout=go.Layout(
        title='Bar Plot for Count',  # Plot title
        xaxis=dict(title='Category'),  # x-axis title
        yaxis=dict(title='Count'),  # y-axis title
        template='plotly'  # Template for styling
    )
)

fig.show()


In [70]:
import plotly.graph_objects as go
import pandas as pd

# Assuming the provided data is already in a DataFrame
data = pd.DataFrame({'count': [334]})

# Create the Plotly Figure
fig = go.Figure(
    data=[
        go.Scatter(
            x=data.index, 
            y=data['count'], 
            mode='markers', 
            marker=dict(color='blue', symbol='circle'),
            hovertemplate='x: %{x}<br>y: %{y}<extra></extra>'
        )
    ],
    layout=go.Layout(
        xaxis=dict(title='Index'),
        yaxis=dict(title='Count'),
        template='plotly'
    )
)

fig.show()

In [84]:
draw_plot(plot_prompt_2, results[1])

"Given the provided dataset and the requirements, I will recommend using a scatter plot to visualize the relationship between different distance measures since it offers better insight for continuous data. Here’s the Plotly code to create the required visualization:\n\n```python \nimport numpy as np\nimport pandas as pd\nimport plotly.graph_objs as go\n\n# Replace this by the provided data\ndata = pd.DataFrame({\n    'count': [334],\n    'avg_distance': [2191.708719],\n    'min_distance': [1407.23],\n    'max_distance': [2958.85]\n})\n\n# Create the Plotly Figure\nfig = go.Figure(\n    data=[\n        go.Scatter(\n            x=['avg_distance', 'min_distance', 'max_distance'], \n            y=[data['avg_distance'][0], data['min_distance'][0], data['max_distance'][0]], \n            mode='markers', \n            marker=dict(color='blue', symbol='circle', size=12),\n            hovertemplate='Distance: %{x}<br>Value: %{y}<extra></extra>'\n        )\n    ],\n    layout={\n        'xaxis':

In [85]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go

# Replace this by the provided data
data = pd.DataFrame({
    'count': [334],
    'avg_distance': [2191.708719],
    'min_distance': [1407.23],
    'max_distance': [2958.85]
})

# Create the Plotly Figure
fig = go.Figure(
    data=[
        go.Scatter(
            x=['avg_distance', 'min_distance', 'max_distance'], 
            y=[data['avg_distance'][0], data['min_distance'][0], data['max_distance'][0]], 
            mode='markers', 
            marker=dict(color='blue', symbol='circle', size=12),
            hovertemplate='Distance: %{x}<br>Value: %{y}<extra></extra>'
        )
    ],
    layout={
        'xaxis': {'title': 'Distance Type'},
        'yaxis': {'title': 'Value'},
        'template': 'plotly',
        'title': {'text': 'Distance Metrics Visualization'}
    }
)

fig.show()

In [86]:
draw_plot(plot_prompt, results[2])

"Certainly! Here's the Python code based on your requirements. We'll use the Plotly library to generate a scatter plot visualization:\n\n```python\nimport pandas as pd\nimport plotly.graph_objects as go\n\n# Example dataset\ndata = pd.DataFrame({\n    'distance_bin': [1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900],\n    'count': [10, 11, 11, 16, 13, 28, 25, 43, 47, 42, 34, 19, 15, 11, 7, 2]\n})\n\n# Create the Plotly Figure\nfigure = go.Figure(\n    data=[\n        go.Scatter(\n            x=data['distance_bin'],\n            y=data['count'],\n            mode='markers',\n            marker=dict(\n                color='blue',\n                symbol='circle'\n            ),\n            hovertemplate='Distance: %{x}<br>Count: %{y}<extra></extra>'\n        )\n    ],\n    layout=dict(\n        title='Scatter Plot of Distance vs Count',\n        xaxis=dict(title='Distance Bin'),\n        yaxis=dict(title='Count'),\n        template='plotly

In [87]:
import pandas as pd
import plotly.graph_objects as go

# Example dataset
data = pd.DataFrame({
    'distance_bin': [1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900],
    'count': [10, 11, 11, 16, 13, 28, 25, 43, 47, 42, 34, 19, 15, 11, 7, 2]
})

# Create the Plotly Figure
figure = go.Figure(
    data=[
        go.Scatter(
            x=data['distance_bin'],
            y=data['count'],
            mode='markers',
            marker=dict(
                color='blue',
                symbol='circle'
            ),
            hovertemplate='Distance: %{x}<br>Count: %{y}<extra></extra>'
        )
    ],
    layout=dict(
        title='Scatter Plot of Distance vs Count',
        xaxis=dict(title='Distance Bin'),
        yaxis=dict(title='Count'),
        template='plotly'
    )
)

figure.show()

In [47]:
import pandas as pd
import plotly.graph_objects as go

# Create the DataFrame
data = pd.DataFrame({
    'distance_bin': [1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900],
    'count': [10, 11, 11, 16, 13, 28, 25, 43, 47, 42, 34, 19, 15, 11, 7, 2]
})

# Create the Plotly Figure
figure = go.Figure(
    data=[
        go.Scatter(
            x=data['distance_bin'],
            y=data['count'],
            mode='markers',
            marker=dict(
                color='blue',
                symbol='circle'
            ),
            hovertemplate='Distance Bin: %{x}<br>Count: %{y}<extra></extra>'
        )
    ],
    layout=dict(
        title='Distance Bin vs. Count',
        xaxis_title='Distance Bin',
        yaxis_title='Count',
        template='plotly'
    )
)

figure.show()

import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.bar('Endpoints', results[0], color='blue')

ax.set_xlabel('Table')
ax.set_ylabel('Count')
ax.set_title('Number of Endpoints in Database')

# Show the plot
plt.show()

In [18]:
print(type(results[0]))

<class 'pandas.core.frame.DataFrame'>


import altair as alt
import pandas as pd

# Example dataset
#data = pd.DataFrame({'X': [1, 2, 3, 4, 5], 'Y': [10, 20, 15, 25, 30]})
data = results[0]

# Altair scatter plot
chart = alt.Chart(data).mark_circle(size=100).encode(
    x='X',
    y='Y',
    tooltip=['X', 'Y']
).properties(title='Altair Scatter Plot')
chart.display()

In [20]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

# Example dataset
data = {'Category': ['A', 'B', 'C', 'D'], 'Values': [10, 20, 15, 25]}
#df = results[0]#
df=pd.DataFrame(data)
print(df)
# Bar plot
sns.set(style="whitegrid")
plt.figure(figsize=(8, 6))
sns.barplot(x='', y='Category', data=df, palette='viridis')
plt.title('Bar Plot')
plt.show()

sns.barplot(x='Metric', y='Value', data=data, palette='Blues_d')
plt.title('Endpoint Count Visualization')
plt.ylabel('Count')
plt.xlabel('Metric')


  Category  Values
0        A      10
1        B      20
2        C      15
3        D      25


ValueError: Could not interpret value `` for `x`. An entry with this name does not appear in `data`.

<Figure size 800x600 with 0 Axes>

In [56]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
##****important run this before run figure.show()
import plotly.io as pio
pio.renderers.default = 'iframe'
import plotly.graph_objects as go



In [30]:
import pandas as pd
#import plotly.graph_objects as go

# Recreate the dataset
data = pd.DataFrame({'ArtistId': [90], 'Name': ['Iron Maiden'], 'PlaylistCount': [516]})

# Extract and render the Plotly Figure
figure = go.Figure(
    data=[
        go.Scatter(
            x=data['ArtistId'],
            y=data['PlaylistCount'],
            mode='markers',
            marker=dict(color='#636efa', symbol='circle'),
            hovertemplate='ArtistId=%{x}<br>PlaylistCount=%{y}<extra></extra>'
        )
    ],
    layout=dict(
        xaxis=dict(title='ArtistId'),
        yaxis=dict(title='PlaylistCount'),
        margin=dict(t=60),
        template='plotly'
    )
)

# Display the figure
figure.show()


In [31]:
import pandas as pd
import plotly.graph_objects as go


In [32]:
# Recreate the dataset
data = pd.DataFrame({'ArtistId': [90], 'Name': ['Iron Maiden'], 'PlaylistCount': [516]})

# Extract and render the Plotly Figure
figure = go.Figure(
    data=[
        go.Scatter(
            x=data['ArtistId'],
            y=data['PlaylistCount'],
            mode='markers',
            marker=dict(color='#636efa', symbol='circle'),
            hovertemplate='ArtistId=%{x}<br>PlaylistCount=%{y}<extra></extra>'
        )
    ],
    layout=dict(
        xaxis=dict(title='ArtistId'),
        yaxis=dict(title='PlaylistCount'),
        margin=dict(t=60),
        template='plotly'
    )
)
# Display the figure
figure.show()

In [33]:
#Test plot code
import plotly.graph_objects as go

# Example plot
fig = go.Figure(
    data=[go.Scatter(x=[1, 2, 3], y=[10, 20, 30], mode='lines')],
    layout=dict(title="Test Plot")
)
fig.show()


In [35]:
# Example dataset
data = {'Category': ['A', 'B', 'C', 'D'], 'Values': [10, 20, 15, 25]}
colors = ['#FF5733', '#33FF57', '#3357FF', '#F3FF33']
fig = go.Figure(
    data=[
        go.Bar(
            x=data['Category'],  # Categories for x-axis
            y=data['Values'],    # Values for y-axis
            name='Category Values',  # Legend name
            #marker=dict(color='skyblue')  # Custom bar color
            marker=dict(color=colors)
        )
    ],
    layout=go.Layout(
        title="Category vs. Values",  # Title of the chart
        xaxis=dict(title="Category"),  # X-axis title
        yaxis=dict(title="Values"),    # Y-axis title
    )
)

fig.show()


In [45]:
import numpy as np

 
fig = go.Figure({
     'data': [{'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
               'hovertemplate': 'Genre=%{label}<br>TotalTracks=%{value}<extra></extra>',
               'labels': np.array(['Alternative', 'Alternative & Punk', 'Blues', 'Bossa Nova', 'Classical',
                                'Comedy', 'Drama', 'Easy Listening', 'Electronica/Dance', 'Heavy Metal',
                                'Hip Hop/Rap', 'Jazz', 'Latin', 'Metal', 'Opera', 'Pop', 'R&B/Soul',
                                'Reggae', 'Rock', 'Rock And Roll', 'Sci Fi & Fantasy',
                                'Science Fiction', 'Soundtrack', 'TV Shows', 'World'], dtype=object),
               'legendgroup': '',
               'name': '',
               'showlegend': True,
               'type': 'pie',
               'values': np.array([  40,  332,   81,   15,   74,   17,   64,   24,   30,   28,   35,  130,
                                 579,  374,    1,   48,   61,   58, 1297,   12,   26,   13,   43,   93,
                                  28])}],
     'layout': {'legend': {'tracegroupgap': 0}, 'template': 'plotly', 'title': {'text': 'Distribution of Genre in the Track'}}
 })
fig.show()

In [58]:
 

fig = go.Figure({
     'data': [{'hovertemplate': 'ArtistId=%{x}<br>PlaylistCount=%{y}<extra></extra>',
               'legendgroup': '',
               'marker': {'color': '#636efa', 'symbol': 'circle'},
               'mode': 'markers',
               'name': '',
               'orientation': 'v',
               'showlegend': False,
               'type': 'scatter',
               'x': np.array([90]),
               'xaxis': 'x',
               'y': np.array([516]),
               'yaxis': 'y'}],
     'layout': {'legend': {'tracegroupgap': 0},
                'margin': {'t': 60},
                'template': 'plotly',
                'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'ArtistId'}},
                'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'PlaylistCount'}}}
 })

fig.show()
#'ggplot2', 'seaborn', 'simple_white', 'plotly', 'plotly_white'